In [1]:
import pandas as pd
import csv
import numpy as np
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
from numpy import mat
from math import gamma, pi, sqrt
from tqdm import tnrange, tqdm_notebook
from time import sleep
from scipy.stats import ttest_ind
from sklearn import linear_model
import time
%matplotlib inline
from WindPy import *
import datetime

In [2]:
w.start()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2017 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

In [3]:
stocks = "000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,000012.SZ,000014.SZ, \
000016.SZ,000017.SZ,000018.SZ,000019.SZ,000020.SZ,000021.SZ,000022.SZ,000023.SZ,000025.SZ,000026.SZ,000027.SZ,000028.SZ,000029.SZ, \
000030.SZ,000031.SZ,000032.SZ,000034.SZ,000035.SZ,000036.SZ,000037.SZ,000038.SZ,000039.SZ,000040.SZ,000042.SZ,000043.SZ,000045.SZ, \
000046.SZ,000048.SZ,000049.SZ,000050.SZ"
date = w.tdays("2017-12-29", "2018-11-30", "Period=M")
dates = date.Times
month = []
for i in range (0,len(dates)):
    aaa = dates[i].strftime("%Y%m%d")
    month.append(aaa)

print(month)

['20171229', '20180131', '20180228', '20180330', '20180427', '20180531', '20180629', '20180731', '20180831', '20180928', '20181031', '20181130']


In [16]:
factors = []
for i in range(0,len(month)):
    factor1=w.wss(stocks, "pct_chg,mkt_cap_ard,swing","tradeDate=" +month[i]+";cycle=M;unit=1")
    factor=pd.DataFrame(factor1.Data)
    factors.append(factor)


In [232]:
hs300 = w.wsd("000300.SH", "pct_chg", "2017-12-29", "2018-11-30", "Period=M;PriceAdj=F")
base = hs300.Data[0]

In [319]:
class Test(object):
    #因子有效性检验
    def __init__(self,month,stocks,factors):
        self.month = month
        self.stocks = stocks
        self.factors = factors
    
    def weight(self,t):
        df = pd.read_csv('weight.csv')
        d = np.array(df['Date']) # dates
        d = d[1:]
        codes = np.array(df.columns[1:])
        weight = np.array(df[df.columns[1:]])
        ww=mat(np.diag(weight[t]))
        return ww
    
    def t_stat(self,t,z):         
        ret1 = list(self.factors[t].T[0])
        ret = mat(ret1).T
        factor1 = list(self.factors[t].T[z])
        factor = mat(factor1).T
        t = abs((ttest_ind(ret,factor)).statistic)
        return t
    
    def ave(self,z):         #1
        t_s1 = []
        for i in range(0,len(month)):
            t=self.t_stat(i,z)[0]
            t_s1.append(t)
        ave=np.array(t_s1).mean()
        return ave
    
    def perct(self,z):          #2
        t_s1 = []
        larger2 = 0
        for i in range(0,len(month)):
            t=self.t_stat(i,z)[0]
            t_s1.append(t)
        for j in range(0,len(t_s1)):
            if t_s1[i]>0:
                larger2 +=1
        percentage = larger2/len(t_s1)
        return percentage

    def return_1(self,t,z):       
        ret1 = list(self.factors[t].T[0])
        ret = mat(ret1).T
        factor1 = list(self.factors[t].T[z])
        factor = mat(factor1).T
        W=self.weight(t)
        f=(factor.T*W*factor).I*factor.T*W*ret
        return f
    
    def return_(self,z):
        output = []
        for i in range (0,len(month)):
            F = self.return_1(i,z)
            output.append(F)
        output1=np.hstack(j for j in output)
        return(output,output1)
    
    def annual_r(self,z):         #3
        ar=1
        m1,m2=self.return_(z)
        for i in range (0,len(m1)):
            ar *= (1+m1[i])
        a_r = ar-1        
        return a_r
    
    def vol(self,z):          #4
        m1,m2=self.return_(z)
        vol = np.std(m1)
        return mat(vol)
    
    def rate(self,z):     #5
        r = self.annual_r(z)/self.vol(z)
        return r
    
    def cor(self,z):      #6
        m1,m2 = self.return_(z)
        m = m2.tolist()
        cor = np.corrcoef(m,base)
        return mat(cor[0,1])
    
    def FSC(self,t,z):     #7
        factor1 = list(self.factors[t].T[z])
        factor = mat(factor1).T
        factor2 = list(self.factors[t+1].T[z])
        factor_ = mat(factor2).T
        W=self.weight(t)
        xbar = np.mean(factor)
        x1bar = np.mean(factor_)
        cal1 = 0
        cal2 = 0
        cal3 = 0
        for i in range (0,len(factor)):
            on = W[i,i]*(factor[i]-xbar)*(factor[i]-x1bar)
            under1 = W[i,i]*(factor[i]-xbar)**2
            under2 = W[i,i]*(factor_[i]-x1bar)**2
            cal1 += on.tolist()[0][0]
            cal2 += under1.tolist()[0][0]
            cal3 += under2.tolist()[0][0]
        rho = cal1/((cal2**(1/2))*(cal3)**(1/2))
        return (rho)    
    
    def VIF(self,z,n):       #8
        factor1 = list(self.factors[0].T[z])
        factor = mat(factor1).T
        factork = []
        for k in range (0,n):
            if k != z:
                fac=list(self.factors[0].T[k])
                factork.append(mat(fac).T)
        output = np.hstack(j for j in factork)
        fk = (output.T*output).I*output.T*factor
        xBar = np.mean(output)
        yBar = np.mean(factor)
        r2 = (fk.T*output.T*factor-len(output)*(yBar**2))/(factor.T*factor-len(output)*(yBar**2))
        VIF = 1/(1-r2)
        return VIF


In [320]:
aaa=Test(month,stocks,factors)
#print(aaa.t_stat(1))
#print(aaa.avet(1))
#print(aaa.perct(1))
#print(aaa.return_(1))
#print(aaa.annual_r(1))
#print(aaa.t_stat(0,1)[0])
#print(aaa.ave(1))
#print(aaa.return_(2))
#print(aaa.vol(2))
#print(aaa.annual_r(2))
#print(aaa.VIF(1,3))
#print(aaa.cor(2))
print(aaa.FSC(0,1))

0.8443435948091983


In [ ]:
weight = []
w_all =0

for i in range(0,2):
    x=factors[i].T[1]
    c=[]
    for j in range(0,42):
        a = (x[j])**(1/2)
        c.append(a)
        z=pd.DataFrame(c)
    w_all = a+w_all
print(w_all)
print(z)
    #print(weight[0])